# 🧪🔗 Testes de Integração - NeuroTranslator PT-EN

Este notebook implementa **testes de integração** e **sistema completo** do NeuroTranslator:
- **Testes unitários** de cada componente
- **Testes de integração** entre módulos
- **Testes de performance** e latência
- **Sistema completo** integrado
- **Simulação de uso real** com dados de exemplo
- **Métricas de qualidade** e monitoramento

## Arquitetura de Testes:
```
┌─────────────────────────────────────────┐
│           TESTES INTEGRADOS             │
├─────────────────┬───────────────────────┤
│  🎤 Áudio       │  📹 Vídeo            │
│  Tests          │  Tests               │
├─────────────────┼───────────────────────┤
│  🌐 Translation │  🔊 TTS              │
│  Tests          │  Tests               │
├─────────────────┼───────────────────────┤
│  📺 Subtitles   │  🖥️ Interface        │
│  Tests          │  Tests               │
├─────────────────────────────────────────┤
│        🚀 SISTEMA COMPLETO              │
└─────────────────────────────────────────┘
```

In [ ]:
# Importações para testes
import unittest
import pytest
import numpy as np
import cv2
import time
import threading
import queue
import json
import os
import sys
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Importar componentes do sistema
sys.path.append('../src')

# Métricas e monitoramento
import psutil
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque

print("🧪 Bibliotecas de teste carregadas!")
print("📊 Sistema de métricas ativo!")

## 1. Sistema de Métricas e Monitoramento

In [ ]:
class PerformanceMonitor:
    """
    Monitor de performance do sistema
    """
    def __init__(self):
        self.metrics = defaultdict(list)
        self.start_time = time.time()
        self.is_monitoring = False
        
    def start_monitoring(self):
        """
        Inicia monitoramento
        """
        self.is_monitoring = True
        self.monitor_thread = threading.Thread(target=self._monitor_loop)
        self.monitor_thread.daemon = True
        self.monitor_thread.start()
        print("📊 Monitoramento iniciado")
    
    def stop_monitoring(self):
        """
        Para monitoramento
        """
        self.is_monitoring = False
        print("📊 Monitoramento parado")
    
    def _monitor_loop(self):
        """
        Loop de monitoramento
        """
        while self.is_monitoring:
            try:
                # CPU e Memória
                cpu_percent = psutil.cpu_percent(interval=1)
                memory = psutil.virtual_memory()
                
                # Timestamp
                timestamp = time.time() - self.start_time
                
                # Armazenar métricas
                self.metrics['timestamp'].append(timestamp)
                self.metrics['cpu_percent'].append(cpu_percent)
                self.metrics['memory_percent'].append(memory.percent)
                self.metrics['memory_used_mb'].append(memory.used / 1024 / 1024)
                
                # Manter apenas últimos 100 pontos
                for key in self.metrics:
                    if len(self.metrics[key]) > 100:
                        self.metrics[key] = self.metrics[key][-100:]
                
            except Exception as e:
                print(f"❌ Erro no monitoramento: {e}")
            
            time.sleep(1)
    
    def add_custom_metric(self, name, value):
        """
        Adiciona métrica customizada
        """
        timestamp = time.time() - self.start_time
        self.metrics[f'{name}_timestamp'].append(timestamp)
        self.metrics[name].append(value)
    
    def get_stats(self):
        """
        Retorna estatísticas
        """
        stats = {}
        
        for key, values in self.metrics.items():
            if values and not key.endswith('_timestamp'):
                stats[key] = {
                    'mean': np.mean(values),
                    'max': np.max(values),
                    'min': np.min(values),
                    'std': np.std(values),
                    'count': len(values)
                }
        
        return stats
    
    def plot_metrics(self):
        """
        Plota métricas de performance
        """
        if not self.metrics['timestamp']:
            print("⚠️ Nenhuma métrica para plotar")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('📊 Métricas de Performance - NeuroTranslator', fontsize=16)
        
        # CPU
        axes[0, 0].plot(self.metrics['timestamp'], self.metrics['cpu_percent'], 'b-')
        axes[0, 0].set_title('🖥️ CPU Usage (%)')
        axes[0, 0].set_xlabel('Tempo (s)')
        axes[0, 0].set_ylabel('CPU %')
        axes[0, 0].grid(True)
        
        # Memória
        axes[0, 1].plot(self.metrics['timestamp'], self.metrics['memory_percent'], 'r-')
        axes[0, 1].set_title('🧠 Memory Usage (%)')
        axes[0, 1].set_xlabel('Tempo (s)')
        axes[0, 1].set_ylabel('Memory %')
        axes[0, 1].grid(True)
        
        # Latência de tradução (se disponível)
        if 'translation_latency' in self.metrics:
            axes[1, 0].plot(self.metrics['translation_latency_timestamp'], 
                           self.metrics['translation_latency'], 'g-')
            axes[1, 0].set_title('⚡ Translation Latency (ms)')
            axes[1, 0].set_xlabel('Tempo (s)')
            axes[1, 0].set_ylabel('Latência (ms)')
            axes[1, 0].grid(True)
        
        # Throughput (se disponível)
        if 'throughput' in self.metrics:
            axes[1, 1].plot(self.metrics['throughput_timestamp'], 
                           self.metrics['throughput'], 'm-')
            axes[1, 1].set_title('🚀 Throughput (translations/min)')
            axes[1, 1].set_xlabel('Tempo (s)')
            axes[1, 1].set_ylabel('Traduções/min')
            axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.show()

print("📊 Sistema de monitoramento implementado!")

## 2. Testes Unitários dos Componentes

In [ ]:
class TestAudioProcessor(unittest.TestCase):
    """
    Testes para processamento de áudio
    """
    
    def setUp(self):
        """Setup para testes"""
        self.sample_rate = 16000
        self.duration = 2  # segundos
        
    def test_audio_generation(self):
        """Testa geração de áudio simulado"""
        # Gerar áudio simulado
        t = np.linspace(0, self.duration, self.sample_rate * self.duration)
        frequency = 440  # Lá (A4)
        audio_data = np.sin(2 * np.pi * frequency * t)
        
        # Verificações
        self.assertEqual(len(audio_data), self.sample_rate * self.duration)
        self.assertTrue(np.all(audio_data >= -1.0))
        self.assertTrue(np.all(audio_data <= 1.0))
        
        print("✅ Teste de geração de áudio passou")
    
    def test_audio_preprocessing(self):
        """Testa pré-processamento de áudio"""
        # Áudio com ruído
        t = np.linspace(0, 1, self.sample_rate)
        signal = np.sin(2 * np.pi * 440 * t)
        noise = np.random.normal(0, 0.1, len(signal))
        noisy_audio = signal + noise
        
        # Normalização
        normalized = noisy_audio / np.max(np.abs(noisy_audio))
        
        # Verificações
        self.assertTrue(np.max(np.abs(normalized)) <= 1.0)
        self.assertGreater(len(normalized), 0)
        
        print("✅ Teste de pré-processamento de áudio passou")
    
    def test_speech_detection(self):
        """Testa detecção de fala simulada"""
        # Simular detecção de fala
        audio_chunk = np.random.normal(0, 0.5, 1024)
        
        # Calcular energia
        energy = np.sum(audio_chunk ** 2)
        threshold = 0.1
        
        has_speech = energy > threshold
        
        # Verificação
        self.assertIsInstance(has_speech, bool)
        
        print("✅ Teste de detecção de fala passou")


class TestTranslationEngine(unittest.TestCase):
    """
    Testes para motor de tradução
    """
    
    def setUp(self):
        """Setup para testes"""
        self.test_phrases = [
            ("Olá, como você está?", "Hello, how are you?"),
            ("Bom dia!", "Good morning!"),
            ("Obrigado pela ajuda", "Thank you for the help"),
            ("Hello, nice to meet you", "Olá, prazer em conhecê-lo"),
            ("Good afternoon", "Boa tarde")
        ]
    
    def test_translation_simulation(self):
        """Testa simulação de tradução"""
        for original, expected in self.test_phrases:
            # Simular tradução (lookup table)
            translation_dict = dict(self.test_phrases)
            
            if original in translation_dict:
                translated = translation_dict[original]
            else:
                # Tradução reversa
                reverse_dict = {v: k for k, v in self.test_phrases}
                translated = reverse_dict.get(original, "Translation not found")
            
            # Verificações
            self.assertIsInstance(translated, str)
            self.assertGreater(len(translated), 0)
        
        print("✅ Teste de simulação de tradução passou")
    
    def test_language_detection(self):
        """Testa detecção de idioma simulada"""
        portuguese_words = ['olá', 'como', 'você', 'está', 'obrigado', 'bom', 'dia']
        english_words = ['hello', 'how', 'are', 'you', 'thank', 'good', 'morning']
        
        def detect_language(text):
            text_lower = text.lower()
            pt_score = sum(1 for word in portuguese_words if word in text_lower)
            en_score = sum(1 for word in english_words if word in text_lower)
            
            if pt_score > en_score:
                return 'pt'
            elif en_score > pt_score:
                return 'en'
            else:
                return 'unknown'
        
        # Testes
        self.assertEqual(detect_language("Olá, como você está?"), 'pt')
        self.assertEqual(detect_language("Hello, how are you?"), 'en')
        
        print("✅ Teste de detecção de idioma passou")
    
    def test_translation_quality(self):
        """Testa qualidade da tradução simulada"""
        def calculate_bleu_score(reference, candidate):
            """BLEU score simplificado"""
            ref_words = reference.lower().split()
            cand_words = candidate.lower().split()
            
            if not cand_words:
                return 0.0
            
            # Precisão de unigramas
            matches = sum(1 for word in cand_words if word in ref_words)
            precision = matches / len(cand_words)
            
            return precision
        
        # Testar com frases conhecidas
        original = "Hello, how are you?"
        reference = "Olá, como você está?"
        candidate = "Olá, como você está?"  # Tradução perfeita
        
        score = calculate_bleu_score(reference, candidate)
        
        self.assertGreaterEqual(score, 0.0)
        self.assertLessEqual(score, 1.0)
        
        print(f"✅ Teste de qualidade passou (BLEU: {score:.2f})")


class TestVideoProcessor(unittest.TestCase):
    """
    Testes para processamento de vídeo
    """
    
    def test_frame_generation(self):
        """Testa geração de frames simulados"""
        # Gerar frame simulado
        height, width = 480, 640
        frame = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Adicionar conteúdo
        cv2.rectangle(frame, (100, 100), (200, 200), (255, 0, 0), -1)
        cv2.circle(frame, (320, 240), 50, (0, 255, 0), -1)
        
        # Verificações
        self.assertEqual(frame.shape, (height, width, 3))
        self.assertEqual(frame.dtype, np.uint8)
        self.assertTrue(np.any(frame > 0))  # Tem conteúdo
        
        print("✅ Teste de geração de frames passou")
    
    def test_face_detection_simulation(self):
        """Testa simulação de detecção facial"""
        # Simular detecção de face
        frame_width, frame_height = 640, 480
        
        # Coordenadas simuladas de face
        face_x, face_y = 200, 150
        face_w, face_h = 100, 120
        
        # Verificar se face está dentro do frame
        face_in_bounds = (
            0 <= face_x < frame_width and
            0 <= face_y < frame_height and
            face_x + face_w <= frame_width and
            face_y + face_h <= frame_height
        )
        
        self.assertTrue(face_in_bounds)
        
        print("✅ Teste de detecção facial simulada passou")
    
    def test_frame_processing(self):
        """Testa processamento de frames"""
        # Frame original
        original_frame = np.random.randint(0, 256, (480, 640, 3), dtype=np.uint8)
        
        # Redimensionar
        resized = cv2.resize(original_frame, (320, 240))
        
        # Converter para escala de cinza
        gray = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)
        
        # Verificações
        self.assertEqual(resized.shape, (240, 320, 3))
        self.assertEqual(gray.shape, (480, 640))
        self.assertEqual(len(gray.shape), 2)  # Escala de cinza
        
        print("✅ Teste de processamento de frames passou")

print("🧪 Testes unitários implementados!")

## 3. Testes de Integração

In [ ]:
class IntegrationTestSuite:
    """
    Suite de testes de integração
    """
    
    def __init__(self):
        self.monitor = PerformanceMonitor()
        self.test_results = []
        
    def run_all_tests(self):
        """
        Executa todos os testes de integração
        """
        print("🚀 Iniciando testes de integração...")
        self.monitor.start_monitoring()
        
        tests = [
            self.test_audio_to_translation,
            self.test_video_with_audio,
            self.test_translation_to_tts,
            self.test_subtitle_system,
            self.test_full_pipeline,
            self.test_concurrent_processing,
            self.test_error_handling,
            self.test_performance_under_load
        ]
        
        for test in tests:
            try:
                start_time = time.time()
                result = test()
                end_time = time.time()
                
                self.test_results.append({
                    'test': test.__name__,
                    'result': result,
                    'duration': end_time - start_time,
                    'status': 'PASS' if result else 'FAIL'
                })
                
            except Exception as e:
                self.test_results.append({
                    'test': test.__name__,
                    'result': False,
                    'duration': 0,
                    'status': 'ERROR',
                    'error': str(e)
                })
        
        self.monitor.stop_monitoring()
        self.print_results()
        
    def test_audio_to_translation(self):
        """
        Testa pipeline áudio → tradução
        """
        print("🎤 Testando pipeline áudio → tradução...")
        
        # Simular captura de áudio
        sample_rate = 16000
        duration = 2
        audio_data = np.random.normal(0, 0.1, sample_rate * duration)
        
        # Simular speech-to-text
        recognized_text = "Olá, como você está?"
        
        # Simular tradução
        translated_text = "Hello, how are you?"
        
        # Verificar pipeline
        pipeline_success = (
            len(audio_data) > 0 and
            len(recognized_text) > 0 and
            len(translated_text) > 0 and
            recognized_text != translated_text
        )
        
        # Métricas
        latency = np.random.uniform(100, 500)  # ms
        self.monitor.add_custom_metric('translation_latency', latency)
        
        print(f"   ✅ Pipeline áudio → tradução: {pipeline_success}")
        print(f"   ⚡ Latência: {latency:.1f}ms")
        
        return pipeline_success
    
    def test_video_with_audio(self):
        """
        Testa sincronização vídeo + áudio
        """
        print("📹 Testando sincronização vídeo + áudio...")
        
        # Simular captura de vídeo
        frame_count = 60  # 2 segundos a 30fps
        frames = []
        
        for i in range(frame_count):
            frame = np.random.randint(0, 256, (480, 640, 3), dtype=np.uint8)
            frames.append(frame)
        
        # Simular áudio sincronizado
        audio_samples = np.random.normal(0, 0.1, 32000)  # 2 segundos
        
        # Verificar sincronização
        video_duration = frame_count / 30  # segundos
        audio_duration = len(audio_samples) / 16000  # segundos
        
        sync_diff = abs(video_duration - audio_duration)
        sync_ok = sync_diff < 0.1  # Tolerância de 100ms
        
        print(f"   ✅ Sincronização V+A: {sync_ok}")
        print(f"   📊 Diferença: {sync_diff*1000:.1f}ms")
        
        return sync_ok
    
    def test_translation_to_tts(self):
        """
        Testa pipeline tradução → TTS
        """
        print("🔊 Testando pipeline tradução → TTS...")
        
        # Texto traduzido
        translated_text = "Hello, how are you?"
        
        # Simular TTS
        def simulate_tts(text):
            # Simular processamento TTS
            processing_time = len(text) * 0.05  # 50ms por caractere
            time.sleep(min(processing_time, 0.5))  # Máximo 500ms
            
            # Gerar áudio simulado
            duration = len(text) * 0.1  # 100ms por caractere
            sample_rate = 22050
            audio_length = int(duration * sample_rate)
            audio = np.random.normal(0, 0.1, audio_length)
            
            return audio
        
        # Executar TTS
        start_time = time.time()
        tts_audio = simulate_tts(translated_text)
        tts_time = (time.time() - start_time) * 1000
        
        # Verificar resultado
        tts_success = len(tts_audio) > 0
        
        print(f"   ✅ Pipeline tradução → TTS: {tts_success}")
        print(f"   ⚡ Tempo TTS: {tts_time:.1f}ms")
        
        return tts_success
    
    def test_subtitle_system(self):
        """
        Testa sistema de legendas
        """
        print("📺 Testando sistema de legendas...")
        
        # Simular legendas
        subtitles = [
            {"start": 0.0, "end": 2.5, "text": "Olá, como você está?"},
            {"start": 2.5, "end": 5.0, "text": "Hello, how are you?"},
            {"start": 5.0, "end": 7.5, "text": "Muito bem, obrigado!"},
            {"start": 7.5, "end": 10.0, "text": "Very well, thank you!"}
        ]
        
        # Testar timing das legendas
        current_time = 3.0  # 3 segundos
        
        active_subtitle = None
        for subtitle in subtitles:
            if subtitle["start"] <= current_time <= subtitle["end"]:
                active_subtitle = subtitle
                break
        
        # Verificar sistema
        subtitle_system_ok = (
            active_subtitle is not None and
            len(active_subtitle["text"]) > 0
        )
        
        print(f"   ✅ Sistema de legendas: {subtitle_system_ok}")
        if active_subtitle:
            print(f"   📝 Legenda ativa: '{active_subtitle['text']}'")
        
        return subtitle_system_ok
    
    def test_full_pipeline(self):
        """
        Testa pipeline completo
        """
        print("🚀 Testando pipeline completo...")
        
        pipeline_steps = [
            "Captura de áudio",
            "Speech-to-text",
            "Detecção de idioma",
            "Tradução",
            "Text-to-speech",
            "Exibição de legendas",
            "Captura de vídeo",
            "Detecção facial"
        ]
        
        # Simular execução de cada etapa
        completed_steps = 0
        total_time = 0
        
        for step in pipeline_steps:
            # Simular processamento
            step_time = np.random.uniform(50, 200)  # 50-200ms
            time.sleep(step_time / 1000)  # Converter para segundos
            
            # Simular sucesso (95% de chance)
            success = np.random.random() > 0.05
            
            if success:
                completed_steps += 1
                total_time += step_time
                print(f"   ✅ {step}: {step_time:.1f}ms")
            else:
                print(f"   ❌ {step}: FALHOU")
                break
        
        # Calcular throughput
        if total_time > 0:
            throughput = 60000 / total_time  # traduções por minuto
            self.monitor.add_custom_metric('throughput', throughput)
        
        pipeline_success = completed_steps == len(pipeline_steps)
        
        print(f"   🎯 Pipeline completo: {pipeline_success}")
        print(f"   📊 Etapas completadas: {completed_steps}/{len(pipeline_steps)}")
        print(f"   ⚡ Tempo total: {total_time:.1f}ms")
        
        return pipeline_success
    
    def test_concurrent_processing(self):
        """
        Testa processamento concorrente
        """
        print("🔄 Testando processamento concorrente...")
        
        # Simular múltiplas threads
        num_threads = 4
        results = queue.Queue()
        
        def worker_thread(thread_id):
            try:
                # Simular trabalho
                work_time = np.random.uniform(0.1, 0.5)
                time.sleep(work_time)
                
                # Simular resultado
                result = {
                    'thread_id': thread_id,
                    'work_time': work_time,
                    'success': True
                }
                results.put(result)
                
            except Exception as e:
                results.put({
                    'thread_id': thread_id,
                    'success': False,
                    'error': str(e)
                })
        
        # Iniciar threads
        threads = []
        start_time = time.time()
        
        for i in range(num_threads):
            thread = threading.Thread(target=worker_thread, args=(i,))
            thread.start()
            threads.append(thread)
        
        # Aguardar conclusão
        for thread in threads:
            thread.join()
        
        end_time = time.time()
        
        # Coletar resultados
        thread_results = []
        while not results.empty():
            thread_results.append(results.get())
        
        # Verificar sucesso
        successful_threads = sum(1 for r in thread_results if r['success'])
        concurrent_success = successful_threads == num_threads
        
        print(f"   ✅ Processamento concorrente: {concurrent_success}")
        print(f"   🧵 Threads bem-sucedidas: {successful_threads}/{num_threads}")
        print(f"   ⏱️ Tempo total: {(end_time - start_time)*1000:.1f}ms")
        
        return concurrent_success
    
    def test_error_handling(self):
        """
        Testa tratamento de erros
        """
        print("⚠️ Testando tratamento de erros...")
        
        error_scenarios = [
            "Áudio corrompido",
            "Falha na tradução",
            "Câmera desconectada",
            "Memória insuficiente",
            "Rede indisponível"
        ]
        
        handled_errors = 0
        
        for scenario in error_scenarios:
            try:
                # Simular erro
                if "corrompido" in scenario:
                    raise ValueError("Dados de áudio inválidos")
                elif "tradução" in scenario:
                    raise ConnectionError("Serviço de tradução indisponível")
                elif "câmera" in scenario:
                    raise RuntimeError("Dispositivo de vídeo não encontrado")
                elif "memória" in scenario:
                    raise MemoryError("Memória insuficiente")
                elif "rede" in scenario:
                    raise TimeoutError("Timeout de rede")
                
            except Exception as e:
                # Simular tratamento do erro
                error_handled = True
                fallback_action = "Usar modo offline"
                
                if error_handled:
                    handled_errors += 1
                    print(f"   ✅ {scenario}: {fallback_action}")
                else:
                    print(f"   ❌ {scenario}: Não tratado")
        
        error_handling_success = handled_errors == len(error_scenarios)
        
        print(f"   🛡️ Tratamento de erros: {error_handling_success}")
        print(f"   📊 Erros tratados: {handled_errors}/{len(error_scenarios)}")
        
        return error_handling_success
    
    def test_performance_under_load(self):
        """
        Testa performance sob carga
        """
        print("🔥 Testando performance sob carga...")
        
        # Simular carga alta
        num_requests = 50
        concurrent_limit = 10
        
        start_time = time.time()
        completed_requests = 0
        failed_requests = 0
        
        # Processar em lotes
        for batch_start in range(0, num_requests, concurrent_limit):
            batch_end = min(batch_start + concurrent_limit, num_requests)
            batch_size = batch_end - batch_start
            
            # Simular processamento do lote
            batch_time = np.random.uniform(0.1, 0.3)
            time.sleep(batch_time)
            
            # Simular sucesso/falha
            batch_success_rate = 0.95
            successful_in_batch = int(batch_size * batch_success_rate)
            
            completed_requests += successful_in_batch
            failed_requests += batch_size - successful_in_batch
        
        end_time = time.time()
        total_time = end_time - start_time
        
        # Calcular métricas
        success_rate = completed_requests / num_requests
        throughput = completed_requests / total_time
        avg_latency = (total_time / completed_requests) * 1000 if completed_requests > 0 else 0
        
        # Critérios de sucesso
        performance_ok = (
            success_rate >= 0.90 and  # 90% de sucesso
            throughput >= 10 and      # 10 req/s
            avg_latency <= 500        # 500ms latência média
        )
        
        print(f"   ✅ Performance sob carga: {performance_ok}")
        print(f"   📊 Taxa de sucesso: {success_rate*100:.1f}%")
        print(f"   🚀 Throughput: {throughput:.1f} req/s")
        print(f"   ⚡ Latência média: {avg_latency:.1f}ms")
        
        return performance_ok
    
    def print_results(self):
        """
        Imprime resultados dos testes
        """
        print("\n" + "="*60)
        print("📊 RESULTADOS DOS TESTES DE INTEGRAÇÃO")
        print("="*60)
        
        passed = sum(1 for r in self.test_results if r['status'] == 'PASS')
        failed = sum(1 for r in self.test_results if r['status'] == 'FAIL')
        errors = sum(1 for r in self.test_results if r['status'] == 'ERROR')
        total = len(self.test_results)
        
        print(f"\n📈 RESUMO:")
        print(f"   ✅ Passou: {passed}/{total} ({passed/total*100:.1f}%)")
        print(f"   ❌ Falhou: {failed}/{total} ({failed/total*100:.1f}%)")
        print(f"   ⚠️ Erros: {errors}/{total} ({errors/total*100:.1f}%)")
        
        print(f"\n📋 DETALHES:")
        for result in self.test_results:
            status_icon = {
                'PASS': '✅',
                'FAIL': '❌',
                'ERROR': '⚠️'
            }[result['status']]
            
            print(f"   {status_icon} {result['test']}: {result['status']} ({result['duration']:.2f}s)")
            
            if 'error' in result:
                print(f"      Erro: {result['error']}")
        
        # Estatísticas de performance
        stats = self.monitor.get_stats()
        if stats:
            print(f"\n📊 ESTATÍSTICAS DE PERFORMANCE:")
            for metric, values in stats.items():
                print(f"   {metric}:")
                print(f"      Média: {values['mean']:.2f}")
                print(f"      Máximo: {values['max']:.2f}")
                print(f"      Mínimo: {values['min']:.2f}")
        
        print("\n" + "="*60)

print("🔗 Testes de integração implementados!")

## 4. Sistema Completo Integrado

In [ ]:
class NeuroTranslatorComplete:
    """
    Sistema NeuroTranslator completo integrado
    """
    
    def __init__(self):
        self.is_running = False
        self.monitor = PerformanceMonitor()
        self.translation_count = 0
        self.start_time = None
        
        # Componentes do sistema
        self.audio_processor = None
        self.video_processor = None
        self.translation_engine = None
        self.tts_engine = None
        self.subtitle_system = None
        
        print("🧠 NeuroTranslator Complete inicializado!")
    
    def initialize_components(self):
        """
        Inicializa todos os componentes
        """
        print("🔧 Inicializando componentes...")
        
        try:
            # Simular inicialização de componentes
            components = [
                "Audio Processor",
                "Video Processor", 
                "Translation Engine",
                "TTS Engine",
                "Subtitle System"
            ]
            
            for component in components:
                # Simular tempo de inicialização
                init_time = np.random.uniform(0.1, 0.5)
                time.sleep(init_time)
                print(f"   ✅ {component}: {init_time*1000:.0f}ms")
            
            print("✅ Todos os componentes inicializados!")
            return True
            
        except Exception as e:
            print(f"❌ Erro na inicialização: {e}")
            return False
    
    def start_system(self):
        """
        Inicia o sistema completo
        """
        if self.is_running:
            print("⚠️ Sistema já está rodando!")
            return
        
        print("🚀 Iniciando NeuroTranslator Complete...")
        
        # Inicializar componentes
        if not self.initialize_components():
            print("❌ Falha na inicialização dos componentes")
            return
        
        # Iniciar monitoramento
        self.monitor.start_monitoring()
        
        # Marcar como ativo
        self.is_running = True
        self.start_time = datetime.now()
        
        # Iniciar threads de processamento
        self.processing_thread = threading.Thread(target=self._main_processing_loop)
        self.processing_thread.daemon = True
        self.processing_thread.start()
        
        print("✅ Sistema iniciado com sucesso!")
        print("🎤 Fale algo para começar a tradução...")
    
    def stop_system(self):
        """
        Para o sistema
        """
        if not self.is_running:
            print("⚠️ Sistema já está parado!")
            return
        
        print("⏹️ Parando NeuroTranslator Complete...")
        
        self.is_running = False
        self.monitor.stop_monitoring()
        
        # Estatísticas finais
        if self.start_time:
            runtime = datetime.now() - self.start_time
            print(f"📊 Tempo de execução: {runtime}")
            print(f"📊 Traduções realizadas: {self.translation_count}")
            
            if runtime.total_seconds() > 0:
                rate = self.translation_count / runtime.total_seconds() * 60
                print(f"📊 Taxa: {rate:.1f} traduções/minuto")
        
        print("✅ Sistema parado com sucesso!")
    
    def _main_processing_loop(self):
        """
        Loop principal de processamento
        """
        print("🔄 Loop de processamento iniciado")
        
        # Frases de exemplo para demonstração
        demo_phrases = [
            ("Olá, como você está hoje?", "Hello, how are you today?", "pt", "en"),
            ("Bom dia! Espero que tenha uma ótima semana.", "Good morning! I hope you have a great week.", "pt", "en"),
            ("Obrigado pela sua ajuda com este projeto.", "Thank you for your help with this project.", "pt", "en"),
            ("Hello, it's nice to meet you!", "Olá, é um prazer conhecê-lo!", "en", "pt"),
            ("Good afternoon, how can I help you?", "Boa tarde, como posso ajudá-lo?", "en", "pt"),
            ("Thank you very much for your time.", "Muito obrigado pelo seu tempo.", "en", "pt"),
            ("Que horas são agora?", "What time is it now?", "pt", "en"),
            ("Onde fica a estação de trem?", "Where is the train station?", "pt", "en"),
            ("How much does this cost?", "Quanto custa isso?", "en", "pt"),
            ("Can you help me, please?", "Você pode me ajudar, por favor?", "en", "pt")
        ]
        
        phrase_index = 0
        
        while self.is_running:
            try:
                # Simular intervalo entre traduções
                time.sleep(np.random.uniform(3, 7))  # 3-7 segundos
                
                if not self.is_running:
                    break
                
                # Selecionar frase para traduzir
                original, translation, src_lang, tgt_lang = demo_phrases[phrase_index]
                phrase_index = (phrase_index + 1) % len(demo_phrases)
                
                # Processar tradução completa
                self._process_translation(original, translation, src_lang, tgt_lang)
                
            except Exception as e:
                print(f"❌ Erro no loop de processamento: {e}")
                time.sleep(1)
        
        print("🔄 Loop de processamento finalizado")
    
    def _process_translation(self, original_text, translated_text, src_lang, tgt_lang):
        """
        Processa uma tradução completa
        """
        start_time = time.time()
        timestamp = datetime.now().strftime("%H:%M:%S")
        
        print(f"\n🔄 [{timestamp}] Processando tradução...")
        
        try:
            # 1. Captura de áudio (simulada)
            audio_time = np.random.uniform(50, 150)
            time.sleep(audio_time / 1000)
            print(f"   🎤 Áudio capturado: {audio_time:.0f}ms")
            
            # 2. Speech-to-text
            stt_time = np.random.uniform(100, 300)
            time.sleep(stt_time / 1000)
            print(f"   🗣️ Speech-to-text: {stt_time:.0f}ms")
            print(f"      Reconhecido: '{original_text}'")
            
            # 3. Detecção de idioma\n            lang_detect_time = np.random.uniform(10, 50)\n            time.sleep(lang_detect_time / 1000)\n            print(f\"   🌐 Detecção de idioma: {lang_detect_time:.0f}ms\")\n            \n            # 4. Tradução\n            translation_time = np.random.uniform(200, 500)\n            time.sleep(translation_time / 1000)\n            print(f\"   🔄 Tradução: {translation_time:.0f}ms\")\n            print(f\"      Traduzido: '{translated_text}'\")\n            \n            # 5. Text-to-speech\n            tts_time = np.random.uniform(150, 400)\n            time.sleep(tts_time / 1000)\n            print(f\"   🔊 Text-to-speech: {tts_time:.0f}ms\")\n            \n            total_time = capture_time + stt_time + lang_detect_time + translation_time + tts_time\n            print(f\"\\n⏱️ Tempo total: {total_time:.0f}ms\")\n            \n            if total_time < 1000:\n                print(\"✅ Performance excelente! (<1s)\")\n            elif total_time < 2000:\n                print(\"✅ Performance boa! (<2s)\")\n            else:\n                print(\"⚠️ Performance pode ser melhorada (>2s)\")\n            \n            return total_time\n        \n        # Executar teste de performance\n        print(\"🚀 Iniciando teste de performance em tempo real...\\n\")\n        \n        for i in range(3):\n            print(f\"📊 Teste {i+1}/3:\")\n            test_realtime_performance()\n            print(\"\\n\" + \"=\"*50 + \"\\n\")\n            time.sleep(1)\n        \n        print(\"✅ Testes de performance concluídos!\")